In [207]:
import geopandas as gpd
import pandas as pd
import numpy as np

This notebook takes a prepared file that defines synthetic operators and generates some test data.

In [ ]:
seed = 0

if "snakemake" in locals():
    input_path = snakemake.input[0]
    output_path = snakemake.output[0]
    days = snakemake.params["days"]
    operator = snakemake.params["operator"]
    
    if "seed" in snakemake.params:
        seed = snakemake.params["seed"]

else:
    input_path = "../../resources/synthetic.gpkg"
    output_path = "../../results/testing/operator.csv"
    days = 12
    operator = "synthetic1"

In [ ]:
# Initialize RNG
random = np.random.RandomState(seed)

In [ ]:
# Load areas and depots from the GeoPackage file
df_areas = gpd.read_file(input_path, layer = "areas")
df_depots = gpd.read_file(input_path, layer = "depots")

In [ ]:
# Obtain the information for the selected operator
df_depot = df_depots[df_depots["operator"] == operator][["geometry"]]
df_area = df_areas[df_areas["operator"] == operator][["geometry"]]

capacity = df_areas.loc[df_areas["operator"] == operator, "capacity"].values[0]
vehicles = df_areas.loc[df_areas["operator"] == operator, "vehicles"].values[0]
step_length = df_areas.loc[df_areas["operator"] == operator, "step_length"].values[0]

In [ ]:
# Generate random vehicle trajectories
df_output = []

for day in range(1, days + 1):
    for vehicle in range(vehicles):
        df_vehicle = []

        df_next = df_depot.copy()
        df_vehicle.append(df_next["geometry"].values[0])

        for k in range(capacity):
            df_next = df_next.buffer(step_length)
            df_next = gpd.clip(df_area, df_next)

            sample = df_next.sample_points(1, rng = random).values

            if len(sample) == 0: # if depot is too far from operating area
                sample = df_area.sample_points(1, rng = random).values

            location = sample[0]
            df_vehicle.append(location)
            df_next["geometry"] = [location]
        
        df_vehicle = gpd.GeoDataFrame(pd.DataFrame(dict(geometry = df_vehicle)), crs = df_area.crs)
        df_vehicle["vehicle_id"] = "{}:{}".format(operator, vehicle)
        df_vehicle["date"] = "2025-01-{:02d}".format(day)
        df_output.append(df_vehicle)

df_output = pd.concat(df_output)

In [ ]:
# Convert to the requested format
df_output = df_output.to_crs("EPSG:4326")

df_output["longitude"] = df_output["geometry"].x
df_output["latitude"] = df_output["geometry"].y

df_output = df_output[["vehicle_id", "date", "longitude", "latitude"]]

In [ ]:
# Output
df_output.to_csv(output_path)